# Advent of Code 2024

--- Day 5: Print Queue ---

Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

```
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13
```
```
75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
```
The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
53 is correctly fourth because it is before page number 29 (53|29).
29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

```
75,47,61,53,29
97,61,53,29,13
75,29,13
```

These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?

In [9]:
# Parse the input file
with open('input/day_5.txt') as f:
    lines = f.read().splitlines()

def parse_rules(rules):
    rules_combined = {}
    # flatten multiple rules into a single list while preserving the order
    for rule in rules:
        left, right = [int(x) for x in rule.split('|')]
        if left not in rules_combined:
            rules_combined[left] = set()
        rules_combined[left].add(right)
    return rules_combined

def parse_updates(updates, parsed_rules):
    sum = 0
    for update in updates:
        update = [int(x) for x in update.split(',')]
        sum += process_print(parsed_rules, update)
    print(sum)

def process_print(parsed_rules, record):
    read = set()
    # check if the rules are violated
    for n in record:
        if n in parsed_rules and len(parsed_rules[n].intersection(read)) > 0:
            return 0
        read.add(n)
    return record[len(record) // 2]


# code to split the input into rules and updates
rules = []
updates = []
for line in lines:
    if '|' in line:
        rules.append(line)
    elif '' != line:
        updates.append(line)

# print(rules)
# print(updates)

parsed_rule = parse_rules(rules)
parsed_updates = parse_updates(updates,parsed_rule)

4569


--- Part Two ---

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

75,97,47,61,53 becomes 97,75,47,61,53.
61,13,29 becomes 61,29,13.
97,13,75,29,47 becomes 97,75,47,29,13.
After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

In [16]:
# Parse the input file
with open('input/day_5.txt') as f:
    lines = f.read().splitlines()


def findInvalid(record,i, rules):
    # Find the invalid record
    v = record[i]
    if v not in rules:
        return -1
    for j in range(i):
        if record[j] in rules[v]:
            return j
    return -1


def sort(record, rules, step=0):
    # Sort the invalid record as per the rules
    p=-1
    for i in range(len(record)):
        p = findInvalid(record, i, rules)
        if p!=-1:
            record = record[:p]+[record[i]]+record[p:i]+record[i+1:]
            break
    if p==-1:
        return record
    return sort(record, rules, step+1)

def process_print(parsed_rules, record):
    read = set()
    # check if the rules are violated
    for n in record:
        if n in parsed_rules and len(parsed_rules[n].intersection(read)) > 0:
            corrected_record = sort(record, parsed_rules)
            return corrected_record[len(corrected_record) // 2]
        read.add(n)
    return 0


def parse_rules(rules):
    # flatten multiple rules into a single list while preserving the order
    rules_combined = {}
    for rule in rules:
        left, right = [int(x) for x in rule.split('|')]
        if left not in rules_combined:
            rules_combined[left] = set()
        rules_combined[left].add(right)
    return rules_combined

def parse_updates(updates, parsed_rules):
    # Function to parse the updates
    sum = 0
    for update in updates:
        update = [int(x) for x in update.split(',')]
        sum += process_print(parsed_rules, update)
    print(sum)




# code to split the input into rules and updates
rules = []
updates = []
for line in lines:
    if '|' in line:
        rules.append(line)
    elif '' != line:
        updates.append(line)

# print(rules)
# print(updates)

parsed_rule = parse_rules(rules)
parsed_updates = parse_updates(updates,parsed_rule)

6456


--- Day 4: Ceres Search ---
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```
Take a look at the little Elf's word search. How many times does XMAS appear?

In [27]:
def find_word(grid, word):
    def search(x, y, dx, dy):
        for i in range(len(word)):
            if not (0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == word[i]):
                return False
            x += dx
            y += dy
        return True

    directions = [(0, 1), (1, 0), (1, 1), (1, -1), (0, -1), (-1, 0), (-1, -1), (-1, 1)]
    count = 0

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            for dx, dy in directions:
                if search(i, j, dx, dy):
                    count += 1

    return count

# Parse the input file
with open('input/day_4.txt') as f:
    lines = f.read().splitlines()

# Example usage
word = "XMAS"
print(find_word(lines, word))  # Output: 18

18


--- Part Two ---
The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:

```
M.S
.A.
M.S
```
Irrelevant characters have again been replaced with . in the above diagram. Within the X, each MAS can be written forwards or backwards.

Here's the same example from before, but this time all of the X-MASes have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

In this example, an X-MAS appears 9 times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an X-MAS appear?

In [29]:
def find_x_mas_pattern(lines):

    # Check if the cell is part of an X-MAS pattern
    m = len(lines)
    n = len(lines[0])

    def check(r, c):
        if lines[r][c] != 'A':
            return False
        ul = lines[r-1][c-1]
        ur = lines[r-1][c+1]
        dl = lines[r+1][c-1]
        dr = lines[r+1][c+1]
        return sorted([ul, ur, dl, dr]) == ['M', 'M', 'S', 'S'] and ul != dr

    return sum(check(r, c) for r in range(1, m-1) for c in range(1, n-1))

# Parse the input file
with open('input/day_4.txt') as f:
    lines = f.read().splitlines()

print(find_x_mas_pattern(lines))

1854


### --- Day 3: Mull It Over ---

"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like `mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 )` do nothing.

For example, consider the following section of corrupted memory:

`xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))`
Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?

In [12]:
import re

sum = 0

# Define the pattern to match the mul instructions
pattern = re.compile(r'mul\((\d+),(\d+)\)')

# Parse the input file
with open('input/day_3.txt') as f:
    lines = f.readlines()

for line in lines:

    matches = pattern.findall(line)

    extracted_parts = [(int(x), int(y)) for x, y in matches]

    # print(extracted_parts)

    for part in extracted_parts:
        sum += part[0] * part[1]

print('Sum:', sum)


Sum: 170778545


### --- Part Two ---

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

- The do() instruction enables future mul instructions.
- The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

`xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))`

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?

In [21]:
import re

sum = 0
enabled = True


# Define the pattern to match the mul instructions
pattern = re.compile(r'(do\(\)|don\'t\(\)|mul\((\d+),(\d+)\))')

# Parse the input file
with open('input/day_3.txt') as f:
    lines = f.readlines()

for line in lines:
    # print(line)

    # Find all matches in the input string
    matches = pattern.findall(line)
    # print(matches)

    for match in matches:
        if match[0] == "do()":
            enabled = True
        elif match[0] == "don't()":
            enabled = False
        elif "mul" in match[0] and enabled:
            sum += int(match[1]) * int(match[2])

print('Sum:', sum)

Sum: 82868252


### --- Day 2: Red-Nosed Reports ---
Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the Red-Nosed Reindeer nuclear fusion/fission plant appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they still talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many reports, one report per line. Each report is a list of numbers called levels that are separated by spaces. For example:

```python
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```


This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are safe. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

- The levels are either all increasing or all decreasing.
- Any two adjacent levels differ by at least one and at most three.

In the example above, the reports can be found safe or unsafe by checking those rules:

- 7 6 4 2 1: Safe because the levels are all decreasing by 1 or 2.
- 1 2 7 8 9: Unsafe because 2 7 is an increase of 5.
- 9 7 6 2 1: Unsafe because 6 2 is a decrease of 4.
- 1 3 2 4 5: Unsafe because 1 3 is increasing but 3 2 is decreasing.
- 8 6 4 4 1: Unsafe because 4 4 is neither an increase or a decrease.
- 1 3 6 7 9: Safe because the levels are all increasing by 1, 2, or 3.

So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [7]:
safe_count=0
unsafe_count=0

# Parse the input file
with open('input/day_2.txt') as f:
    lines = f.readlines()
levels = [list(map(int, line.split())) for line in lines]
#print(levels)

# Check each report
for report in levels:
    #print('Report:', report)
    safe_flag = True
    score = int()
    increasing = None

    # Check each level in the report with the previous level
    for previousLevel, currentLevel in zip(report, report[1:]):
        # print('Previous:', previousLevel, 'Current:', currentLevel)

        score = currentLevel - previousLevel

        if score < -3 or score > 3 or score == 0:
            safe_flag=False
        if increasing and score < 0:
            safe_flag=False
        if increasing == False and score > 0:
            safe_flag=False

        if score < 0:
            increasing = False
        else:
            increasing = True

    if safe_flag:
        safe_count += 1
    else:
        unsafe_count += 1

print('Safe:', safe_count, 'Unsafe:', unsafe_count)

Safe: 479 Unsafe: 521


### --- Part Two ---

The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- 7 6 4 2 1: Safe without removing any level.
- 1 2 7 8 9: Unsafe regardless of which level is removed.
- 9 7 6 2 1: Unsafe regardless of which level is removed.
- 1 3 2 4 5: Safe by removing the second level, 3.
- 8 6 4 4 1: Safe by removing the third level, 4.
- 1 3 6 7 9: Safe without removing any level.

Thanks to the Problem Dampener, 4 reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

In [11]:
safe_count=0
unsafe_count=0

def checkSafe(report):

    increasing = None

    # Check each level in the report with the previous level
    for previousLevel, currentLevel in zip(report, report[1:]):
        # print('Previous:', previousLevel, 'Current:', currentLevel)

        score = currentLevel - previousLevel

        if score < -3 or score > 3 or score == 0:
            return False
        if increasing and score < 0:
             return False
        if increasing == False and score > 0:
             return False

        if score < 0:
            increasing = False
        else:
            increasing = True

    return True

# Parse the input file
with open('input/day_2.txt') as f:
    lines = f.readlines()
levels = [list(map(int, line.split())) for line in lines]
#print(levels)

for report in levels:
    if checkSafe(report):
        safe_count += 1
    else: # Part 2
        for i, level in enumerate(report):
            reportCopy = report.copy()
            reportCopy.pop(i)
            if checkSafe(reportCopy):
                safe_count += 1
                break

print(safe_count)

531


### --- Day 1: Historian Hysteria ---

Upon pouring into the office, everyone confirms that the Chief Historian is indeed nowhere to be found. Instead, the Elves discover an assortment of notes and lists of historically significant locations! This seems to be the planning the Chief Historian was doing before he left. Perhaps these notes can be used to determine which locations to search?

Throughout the Chief's office, the historically significant locations are listed not by name but by a unique number called the location ID. To make sure they don't miss anything, The Historians split into two groups, each searching the office and trying to create their own complete list of location IDs.

There's just one problem: by holding the two lists up side by side (your puzzle input), it quickly becomes clear that the lists aren't very similar. Maybe you can help The Historians reconcile their lists?

For example:

```python
3   4
4   3
2   5
1   3
3   9
3   3
```
Maybe the lists are only off by a small amount! To find out, pair up the numbers and measure how far apart they are. Pair up the smallest number in the left list with the smallest number in the right list, then the second-smallest left number with the second-smallest right number, and so on.

Within each pair, figure out how far apart the two numbers are; you'll need to add up all of those distances. For example, if you pair up a 3 from the left list with a 7 from the right list, the distance apart is 4; if you pair up a 9 with a 3, the distance apart is 6.

In the example list above, the pairs and distances would be as follows:

- The smallest number in the left list is 1, and the smallest number in the right list is 3. The distance between them is 2.
- The second-smallest number in the left list is 2, and the second-smallest number in the right list is another 3. The distance between them is 1.
- The third-smallest number in both lists is 3, so the distance between them is 0.
- The next numbers to pair up are 3 and 4, a distance of 1.
- The fifth-smallest numbers in each list are 3 and 5, a distance of 2.
- Finally, the largest number in the left list is 4, while the largest number in the right list is 9; these are a distance 5 apart.

To find the total distance between the left list and the right list, add up the distances between all of the pairs you found. In the example above, this is 2 + 1 + 0 + 1 + 2 + 5, a total distance of 11!

Your actual left and right lists contain many location IDs. What is the total distance between your lists?

In [1]:
list1 = []
list2 = []
total_sum = 0
with open('input/day_1.txt') as f:
    lines = f.readlines()
    for line in lines:
        values = line.split()
        list1.append(int(values[0]))
        list2.append(int(values[1]))
    list1.sort()
    list2.sort()
    total_sum = sum(abs(a - b) for a, b in zip(list1, list2))
print(total_sum)

2176849


### --- Part Two ---


Your analysis only confirmed what everyone feared: the two lists of location IDs are indeed very different.

Or are they?

The Historians can't agree on which group made the mistakes or how to read most of the Chief's handwriting, but in the commotion you notice an interesting detail: a lot of location IDs appear in both lists! Maybe the other numbers aren't location IDs at all but rather misinterpreted handwriting.

This time, you'll need to figure out exactly how often each number from the left list appears in the right list. Calculate a total similarity score by adding up each number in the left list after multiplying it by the number of times that number appears in the right list.

Here are the same example lists again:

```python
3   4
4   3
2   5
1   3
3   9
3   3
```
For these example lists, here is the process of finding the similarity score:

- The first number in the left list is 3. It appears in the right list three times, so the similarity score increases by 3 * 3 = 9.
- The second number in the left list is 4. It appears in the right list once, so the similarity score increases by 4 * 1 = 4.
- The third number in the left list is 2. It does not appear in the right list, so the similarity score does not increase (2 * 0 = 0).
- The fourth number, 1, also does not appear in the right list.
- The fifth number, 3, appears in the right list three times; the similarity score increases by 9.
- The last number, 3, appears in the right list three times; the similarity score again increases by 9.

So, for these example lists, the similarity score at the end of this process is 31 (9 + 4 + 0 + 0 + 9 + 9).

Once again consider your left and right lists. What is their similarity score?

In [3]:
from collections import Counter

list1 = []
list2 = []
total_sum = 0
with open('input/day_1.txt') as f:
    lines = f.readlines()
    for line in lines:
        values = line.split()
        list1.append(int(values[0]))
        list2.append(int(values[1]))
    lc  = Counter(list2)
    for l in list1:
        total_sum += l * lc[l]
print(total_sum)

23384288
